# IsotopeQuantity examples

First, imports and isotope definitions to use later.

In [1]:
from becquerel.tools.isotope import Isotope, IsotopeQuantity, NeutronIrradiation
import datetime

cs137 = Isotope('Cs-137')
ba133 = Isotope('Ba-133')
cd109 = Isotope('Cd-109')
co57 = Isotope('Co-57')
co60 = Isotope('Co-60')
mn54 = Isotope('Mn-54')
na22 = Isotope('Na-22')
k40 = Isotope('K-40')
u238 = Isotope('U-238')

## Decay calculations with `IsotopeQuantity`

Let's see how we can use the IsotopeQuantity class to work with lab check sources.

Let's start with the ubiquitous Cs-137 check source. How strong is it now?

In [2]:
cs137_chk = IsotopeQuantity(cs137, date='2008-01-15', uci=11.13)
cs137_chk.uci_now()

8.9717106122138297

In fact, we can input an inventory of sources and easily see what's still active, without looking up half lives.

In [3]:
ba133_chk = IsotopeQuantity(ba133, date='2008-01-15', uci=10.24)
cd109_chk = IsotopeQuantity(cd109, date='2008-01-15', uci=10.3)
co57_chk = IsotopeQuantity(co57, date='2008-01-15', uci=11.21)
co60_chk = IsotopeQuantity(co60, date='2008-01-15', uci=9.38)
mn54_chk = IsotopeQuantity(mn54, date='2008-01-15', uci=9.03)
na22_chk = IsotopeQuantity(na22, date='2008-01-15', uci=10.21)

for chk in (ba133_chk, cd109_chk, co57_chk, co60_chk, mn54_chk, na22_chk):
    print('{}: {:.3f} uCi'.format(chk.isotope, chk.uci_now()))

Ba-133: 5.539 uCi
Cd-109: 0.061 uCi
Co-57: 0.002 uCi
Co-60: 2.740 uCi
Mn-54: 0.005 uCi
Na-22: 0.845 uCi


There are a variety of ways to specify a source activity: Bq, number of atoms, or grams.

In [4]:
IsotopeQuantity(co60, date='2017-03-16 14:00:00', bq=3.7e4)
IsotopeQuantity(k40, date=datetime.datetime.now(), atoms=4.2e24)
IsotopeQuantity(u238, g=0.33)

If the `date` argument is not specified, the reference date defaults to `datetime.datetime.now()`.

Of course, we can check activities at other times too, and check other quantities.

In [5]:
co60_chk.uci_at('2014-11-05')

3.8317515981908059

In [6]:
cd109_chk.bq_now()

2245.9534800162178

In [7]:
cs137_chk.g_at(cs137_chk.ref_date)

1.2830528795547293e-07

In [8]:
co57_chk.atoms_now()

2304691219.0364914

By default, the reference date we give is the creation date of the quantity. So we can't evaluate an activity before then.

In [9]:
cs137_chk.bq_at('1999-01-01')

IsotopeError: The source represented by this IsotopeQuantity was created at 2008-01-15 00:00:00 and thus did not exist at 1999-01-01

But we can change this behavior.

In [10]:
k40_src = IsotopeQuantity(k40, date=datetime.datetime.now(), g=1.0, creation_date=False)
k40_src.g_at('1999-01-01')

1.0000000102165703

We can also evaluate the number of decays in a specific time interval, or the average activity in the interval.

(In the future, this will be useful for interfacing directly with a `Spectrum` object, and calibrating efficiency, for example.)

In [11]:
cs137_chk.decays_from('2017-05-22 12:00:00', '2017-05-22 12:05:00')

99595559.0625

In [12]:
cs137_chk.uci_from('2017-05-22 12:00:00', '2017-05-22 12:05:00')

8.9725728885135148

Finally, we can also calculate times when the source will have decayed to a given level:

In [13]:
print(cs137_chk.time_when(uci=1))

2112-08-12 22:32:16.007576


## Calculating activations with `NeutronIrradiation`

Currently, capture cross sections are not scraped from the web, so for now they must be input by hand. 

Still, `NeutronIrradiation` can make our life easier.

In [14]:
irradiation_start = '2017-04-30 10:32:00'
irradiation_stop = '2017-04-30 11:32:00'
flux = 3.1e11
ni = NeutronIrradiation(irradiation_start, irradiation_stop, n_cm2_s=flux)
print(ni)

3.1e+11 n/cm2/s from 2017-04-30 10:32:00 to 2017-04-30 11:32:00


We can calculate how much of a sample will be activated:

In [15]:
iso = Isotope('Na-23')
sample = IsotopeQuantity(iso, g=1.4, creation_date=False)
iso2 = Isotope('Na-24')
barns = 23   # making this up for now

na24_qty = ni.activate(barns, initial_iso_q=sample, activated_iso=iso2)
print(na24_qty)

11804946779.6 Bq of Na-24 (at 2017-04-30 11:32:00)


We can also back-calculate how much was in the original sample:

In [16]:
activated_qty = IsotopeQuantity(iso2, bq=103.2, creation_date=False)
na23_qty = ni.activate(barns, initial_iso=iso, activated_iso_q=activated_qty)
print(na23_qty)

2693.23197921 g of Na-23
